In [1]:
from tvplayer import TVPlayer
import time

In [4]:
player = TVPlayer(["nbc","abc"])
playlist_file, schedules,info=player.initialize()

In [3]:
player.start()
time.sleep(5)
player.vlc_seek(700)
time.sleep(10)
print("shutdown")
#player.stop()

started


NameError: name 'time' is not defined

In [ ]:
player.channel_up()

In [ ]:
player.stop()

In [40]:
for i in range(0,1):
    print(i)

0


In [83]:
test=[{},{},{}]
for e in test:
    e["test"]=1
print(test)

[{'test': 1}, {'test': 1}, {'test': 1}]


In [ ]:
with Telnet('127.0.0.1',1234) as tn:
    tn.write('1.mp4\n\c'.encode('utf-8'))

In [125]:
test=[1,2,3]
print(test.last())

AttributeError: 'list' object has no attribute 'last'